In [1]:
import codecs,math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Lecture de Lexique3
1. lecture dans un DF *Lexique3*
1. extraction des colonnes pertinentes *lex3*

In [2]:
nomLexiqueOrg="/Users/gilles/ownCloud/Python/phonemisation/Lexique380-UTF8.txt"
nomRepertoire="/Volumes/gilles/Transfert/Copies-iMac-GB/Python/phonemisation/"
nomLexiqueOrg=nomRepertoire+"Lexique380-UTF8.txt"
Lexique3=pd.read_csv(nomLexiqueOrg,sep="\t")

In [3]:
Lexique3.columns

Index([u'1_ortho', u'2_phon', u'3_lemme', u'4_cgram', u'5_genre', u'6_nombre',
       u'7_freqlemfilms2', u'8_freqlemlivres', u'9_freqfilms2',
       u'10_freqlivres', u'11_infover', u'12_nbhomogr', u'13_nbhomoph',
       u'14_islem', u'15_nblettres', u'16_nbphons', u'17_cvcv', u'18_p_cvcv',
       u'19_voisorth', u'20_voisphon', u'21_puorth', u'22_puphon', u'23_syll',
       u'24_nbsyll', u'25_cv-cv', u'26_orthrenv', u'27_phonrenv',
       u'28_orthosyll', u'29_cgramortho', u'30_deflem', u'31_defobs',
       u'32_old20', u'33_pld20', u'34_morphoder', u'35_nbmorph'],
      dtype='object')

In [4]:
lex3=Lexique3[["1_ortho","2_phon","3_lemme","4_cgram","5_genre","6_nombre","7_freqlemfilms2","8_freqlemlivres","9_freqfilms2","10_freqlivres","11_infover"]].copy()
Lexique3=None

In [5]:
lex3.loc[:,"7_freqlemfilms2"]=lex3.loc[:,"7_freqlemfilms2"].astype(float)
lex3.loc[:,"8_freqlemlivres"]=lex3.loc[:,"8_freqlemlivres"].astype(float)
lex3.loc[:,"9_freqfilms2"]=lex3.loc[:,"9_freqfilms2"].astype(float)
lex3.loc[:,"10_freqlivres"]=lex3.loc[:,"10_freqlivres"].astype(float)
lex3[["1_ortho","3_lemme","9_freqfilms2"]].head()

,1_ortho,3_lemme,9_freqfilms2
0,a,a,81.36
1,a,avoir,6350.91
2,a,avoir,5498.34
3,a capella,a capella,0.04
4,a cappella,a cappella,0.04


In [9]:
lex3[lex3["4_cgram"]=="VER"]["9_freqfilms2"].sum()/(lex3["9_freqfilms2"].sum())

0.21868152277204744

## Analyse de Lexique3

#### Ajout du rang pour les formes

In [6]:
lex3.loc[:,"rangFW"]=lex3.loc[:,"9_freqfilms2"].rank(ascending=False)
lex3.loc[:,"rangTW"]=lex3.loc[:,"10_freqlivres"].rank(ascending=False)
lex3.head()

,1_ortho,2_phon,3_lemme,4_cgram,5_genre,6_nombre,7_freqlemfilms2,8_freqlemlivres,9_freqfilms2,10_freqlivres,11_infover,rangFW,rangTW
0,a,a,a,NOM,m,NaN,81.36,58.65,81.36,58.65,NaN,955.0,1297.5
1,a,a,avoir,AUX,NaN,NaN,18559.22,12800.81,6350.91,2926.69,ind:pre:3s;,22.0,46.0
2,a,a,avoir,VER,NaN,NaN,13572.40,6426.49,5498.34,1669.39,ind:pre:3s;,25.0,64.0
3,a capella,akapEla,a capella,ADV,NaN,NaN,0.04,0.07,0.04,0.07,NaN,77026.5,106149.5
4,a cappella,akapEla,a cappella,ADV,NaN,NaN,0.04,0.07,0.04,0.07,NaN,77026.5,106149.5


### Graphes Fréquence=f(Rang)

In [ ]:
%matplotlib inline

#### Choix du style de plot

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_palette("hls")
#sns.palplot(sns.color_palette("hls", 8))
#sns.palplot(sns.color_palette("husl", 8))

#### Distribution de la fréquence des formes dans Films

In [ ]:
ax=lex3.sort_values("rangFW").set_index("rangFW")["9_freqfilms2"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title="Formes dans Films")
None

#### Distribution de la fréquence des formes dans Textes

In [ ]:
ax=lex3.sort_values("rangTW").set_index("rangTW")["10_freqlivres"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title="Formes dans Textes")
None

#### Extraction des fréquences de lexèmes

In [ ]:
lex3Lemmes=lex3.drop_duplicates(subset=["3_lemme","4_cgram","7_freqlemfilms2","8_freqlemlivres"]).copy()
lex3Lemmes.loc[:,"rangFL"]=lex3Lemmes.loc[:,"7_freqlemfilms2"].rank(ascending=False)
lex3Lemmes.loc[:,"rangTL"]=lex3Lemmes.loc[:,"8_freqlemlivres"].rank(ascending=False)
lex3Lemmes.head()

#### Distribution de la fréquence des lexèmes dans Films

In [ ]:
ax=lex3Lemmes.sort_values("rangFL").set_index("rangFL")["7_freqlemfilms2"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title=u"Lexèmes dans Films")
None

#### Distribution de la fréquence des lexèmes dans Textes

In [ ]:
ax=lex3Lemmes.sort_values("rangTL").set_index("rangTL")["8_freqlemlivres"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title=u"Lexèmes dans Textes")
None

## Analyse de Verbes3

### Extraction des verbes de Lexique3

In [ ]:
lex3.rename(columns={'11_infover':'lex3_11_infover'}, inplace=True)

In [ ]:
verb3=lex3[lex3["4_cgram"]=="VER"]

In [ ]:
verb3.loc[:,"rangFWV"]=verb3.loc[:,"9_freqfilms2"].rank(ascending=False)
verb3.loc[:,"rangTWV"]=verb3.loc[:,"10_freqlivres"].rank(ascending=False)
verb3.head()

### Graphes Fréquence=f(Rang)

In [ ]:
ax=verb3.sort_values("rangFWV").set_index("rangFWV")["9_freqfilms2"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title="Formes verbales dans Films")
None

In [ ]:
ax=verb3.sort_values("rangTWV").set_index("rangTWV")["10_freqlivres"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title="Formes verbales dans Textes")
None

#### Extraction des fréquences de lexèmes

In [ ]:
verb3Lemmes=verb3.drop_duplicates(subset=["3_lemme","4_cgram","7_freqlemfilms2","8_freqlemlivres"]).copy()
verb3Lemmes.loc[:,"rangFLV"]=verb3Lemmes.loc[:,"7_freqlemfilms2"].rank(ascending=False)
verb3Lemmes.loc[:,"rangTLV"]=verb3Lemmes.loc[:,"8_freqlemlivres"].rank(ascending=False)
verb3Lemmes.head()

#### Distribution de la fréquence des lexèmes dans Films

In [ ]:
ax=verb3Lemmes.sort_values("rangFLV").set_index("rangFLV")["7_freqlemfilms2"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title=u"Lexèmes verbaux dans Films")
None

In [ ]:
ax=verb3Lemmes.sort_values("rangTLV").set_index("rangTLV")["8_freqlemlivres"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title=u"Lexèmes verbaux dans Textes")
None

# TESTS 
## Examen de la distribution des échantillons

In [ ]:
import pickle

In [ ]:
nomSample="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/Longitudinales/Longitudinal-55-T1100000-F41134.pkl"

In [ ]:
with open(nomSample,"rb") as input:
    dfSample=pickle.load(input)
dfSample["rang"]=dfSample.tir1.rank(ascending=False)

In [ ]:
ax=dfSample.sort_values("rang").set_index("rang")["tir1"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title=u"Formes verbales")
None

In [ ]:
lexemesSample=dfSample.groupby("lexeme")[["tir1"]].sum()
lexemesSample["rang"]=lexemesSample.tir1.rank(ascending=False)

In [ ]:
ax=lexemesSample.sort_values("rang").set_index("rang")["tir1"].plot()
ax.set(xscale="log",yscale="log",xlabel="Rang",ylabel=u"Fréquence",title=u"Verbes")
None

# Projet de thèse Gauvain

In [ ]:
lex3[lex3["7_freqlemfilms2"]>=.1].groupby("3_lemme").count()

In [ ]:
for seuil in [100, 10, 1, 0.1, 0.01]:
    print "seuil",seuil
    dfFilms=lex3[lex3["7_freqlemfilms2"]>=seuil].groupby("3_lemme")
    countFilms=dfFilms.count()
    print "\tFilms"
#    print "\tTextes",lex3[(lex3["8_freqlemlivres"]>=seuil)].groupby("3_lemme").count().shape
#    print "\tLex3",lex3[(lex3["7_freqlemfilms2"]>=seuil)|(lex3["8_freqlemlivres"]>=seuil)].groupby("3_lemme").count().shape

In [ ]:
lex3.groupby("3_lemme").count()

In [14]:
lex3[(lex3["7_freqlemfilms2"]>100)].groupby("3_lemme").count().shape[0]

705

In [15]:
lex3[lex3["7_freqlemfilms2"]>=10].groupby("3_lemme").count().shape[0]

3676

In [16]:
lex3[lex3["7_freqlemfilms2"]>=1].groupby("3_lemme").count().shape[0]

12486

In [7]:
lex3[lex3["7_freqlemfilms2"]>=0.1].groupby("3_lemme").count().shape[0]

26345

In [8]:
lex3[lex3["7_freqlemfilms2"]>=0.01].groupby("3_lemme").count().shape[0]

37630

In [9]:
lex3[(lex3["8_freqlemlivres"]>100)].groupby("3_lemme").count().shape[0]

806

In [10]:
lex3[(lex3["8_freqlemlivres"]>10)].groupby("3_lemme").count().shape[0]

5147

In [11]:
lex3[(lex3["8_freqlemlivres"]>1)].groupby("3_lemme").count().shape[0]

18070

In [12]:
lex3[(lex3["8_freqlemlivres"]>0.1)].groupby("3_lemme").count().shape[0]

32888

In [13]:
lex3[(lex3["8_freqlemlivres"]>0.01)].groupby("3_lemme").count().shape[0]

40120